In [7]:
max_length = 512
seed = 3407

In [8]:
import torch
import random
import numpy as np
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed
seed_everything(seed)

3407

In [9]:
from tqdm import tqdm
paragraphs = []
p = []
with open('paragraph.txt', 'r', encoding='UTF-8') as f:
    total_data = f.readlines()
    with tqdm(total_data) as loader:
        for data in loader:
            if data == "#*#" + '\n':
                paragraphs.append(p.copy())
                p.clear()
            else:
                p.append(data)

len(paragraphs)

100%|██████████| 147/147 [00:00<?, ?it/s]


36

In [10]:
def get_next_sentence_sample(sentence, next_sentence, paragraphs, nsp_probability=0.5):
   if random.random() < nsp_probability:
       labels = 1
   else:
       next_sentence = random.choice(random.choice(paragraphs))
       labels = 0
   return sentence, next_sentence, labels

In [11]:
from datasets import Dataset
dict_dataset = {"sentence":[],"next_sentence":[],"labels":[]}
for p in tqdm(paragraphs):
    for i in range(len(p) - 1):  # 遍历一个段落中的每一句话
        sentence, next_sentence, labels = get_next_sentence_sample(p[i], p[i + 1], paragraphs)  # 构造NSP样本
        if len(sentence)>=500:
            continue
        else:
            dict_dataset["sentence"].append(sentence)
            dict_dataset["next_sentence"].append(next_sentence)
            dict_dataset["labels"].append(labels)

dataset = Dataset.from_dict(dict_dataset)
dataset

100%|██████████| 36/36 [00:00<00:00, 35925.52it/s]


Dataset({
    features: ['sentence', 'next_sentence', 'labels'],
    num_rows: 75
})

In [12]:
model_path = 'bert-base-chinese'
vocab_path = model_path
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(vocab_path)

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000016CBBC2A4F0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


In [13]:
def tokenization(example):
    return tokenizer(example["sentence"],example["next_sentence"],truncation=True, padding="max_length",max_length=max_length)

dataset = dataset.map(tokenization, batched=True)
dataset

Parameter 'function'=<function tokenization at 0x0000016CC22B74C0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/1 [00:00<?, ?ba/s]


Dataset({
    features: ['sentence', 'next_sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 75
})

In [14]:
dataset = dataset.remove_columns(["sentence","next_sentence"])
dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 75
})

In [15]:
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask","labels"])
dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 75
})

In [16]:
dataset=dataset.train_test_split(test_size=0.01,shuffle=True,seed=seed)
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 74
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1
    })
})

In [17]:
dataset.save_to_disk("./dataset_paragraphs")


Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]
